In [2]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import numpy as np

In [64]:
# Google Sheets Connect
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/spreadsheets", "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name('creds.json', scope)
client = gspread.authorize(creds)
sheet = client.open("Handwritten-Dataset").sheet1

df = pd.DataFrame(sheet.get_all_values())
df.columns = df.iloc[0]
df = df[1:]

In [65]:
for i in df.index: 
    df.at[i, "grid"] = df.at[i, "grid"].split(',')

In [147]:
x_data = []
y_data = df["label"]
for i in df.index:
    x_data.append(df.at[i,"grid"])
x_data = np.asarray(x_data).T
x_data = x_data.astype('int32')
y_data = y_data.astype('int32')

m, n = x_data.shape

In [148]:
def init_params():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A
    
def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

In [157]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    # print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            # print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [158]:
W1, b1, W2, b2 = gradient_descent(x_data, y_data, 0.10, 500)

C:\Users\acer\AppData\Local\Temp/ipykernel_12640/3962810796.py:12: RuntimeWarning: overflow encountered in exp
  A = np.exp(Z) / sum(np.exp(Z))
C:\Users\acer\AppData\Local\Temp/ipykernel_12640/3962810796.py:12: RuntimeWarning: invalid value encountered in true_divide
  A = np.exp(Z) / sum(np.exp(Z))


0.0967741935483871
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.0967741935483871
